In [1]:
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2 import robjects
import pandas as pd
import psycopg2
import h2o

In [2]:
def get_best_models(area):
    try:
        connection = psycopg2.connect(
            database="postgres",
            user="renan",
            password="1234567890",
            host="localhost",
            port="5432"
        )

        # Cria um cursor para executar consultas
        cursor = connection.cursor()

        # Executa a consulta
        query = f"""
                SELECT "Tipo", "Subtipo", "MinDocs", "RangeDocs", COUNT(*)
                FROM(
                    SELECT * FROM public.models
                    WHERE ("Subarea", "F1") IN (

                      SELECT "Subarea", MAX("F1") AS max_f1

                      FROM public.models
                      WHERE "Area" = '{area}' AND "Subtipo" != 'ranger'

                      GROUP BY "Subarea"

                    )
                    ORDER BY "F1" DESC
                ) as tabela

                GROUP BY "Tipo", "Subtipo", "MinDocs", "RangeDocs"
                ORDER BY COUNT(*) DESC
                LIMIT 3
                """
        cursor.execute(query)

        # Recupera os resultados da consulta como uma lista de tuplas
        results = cursor.fetchall()

        # Cria um DataFrame pandas com os resultados
        df = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])

        # Fecha o cursor e a conexão
        cursor.close()

    except psycopg2.Error as error:
        print("Erro ao conectar ao PostgreSQL:", error)

    finally:
        # Fecha a conexão com o banco de dados
        if 'connection' in locals():
            connection.close()
        
        return df

In [3]:
def prevNN(abstract):
    localH2o = h2o.init(nthreads = -1)
    
    Modelo = h2o.load_model('Modelos/DeepLearning_model_R_1670582405235_1')
    prevNN = Modelo.predict(h2o.H2OFrame(abstract))
    
    return prevNN[0, 0]

In [4]:
def prevC50(dados, str_modelo):
    base = importr('base')
    utils = importr('utils') 
    C50 = importr('C50')
    
    robjects.r['load']('Modelos/' + str_modelo)
    Modelo = robjects.r['Modelo']
    
    predict = robjects.r('predict')
    
    prevC50 = predict(Modelo, type = "class", newdata = dados)
    
    del C50
    del utils
    del base
    del Modelo
    del predict
    
    return prevC50.levels[prevC50[0]-1]

In [5]:
def prevRFranger(dados, str_modelo):
    base = importr('base')
    utils = importr('utils')    
    ranger = importr('ranger')
    
    robjects.r['load']('Modelos/' + str_modelo)
    Modelo = robjects.r['Modelo']
    
    predict = robjects.r('predict')
    
    prevRF_ranger = predict(Modelo, data = dados)
    prevRF_ranger = prevRF_ranger[0]
    
    del ranger
    del utils
    del base
    del Modelo
    del predict
    
    return prevRF_ranger.levels[prevRF_ranger[0]-1]

In [6]:
def prevRFtrad(dados, str_modelo):
    base = importr('base')
    utils = importr('utils')    
    rf = importr('randomForest')
    
    robjects.r['load']('Modelos/' + str_modelo)
    Modelo = robjects.r['Modelo']
    
    predict = robjects.r('predict')
    
    prevRF_trad = predict(Modelo, data = dados)
    
    del rf
    del utils
    del base
    del Modelo
    del predict
    
    return prevRF_trad.levels[prevRF_trad[0]-1]

In [7]:
def prevRpart(dados, str_modelo):
    base = importr('base')
    utils = importr('utils')    
    rpart= importr('rpart')
    
    robjects.r['load']('Modelos/' + str_modelo)
    Modelo = robjects.r['Modelo']
    
    predict = robjects.r('predict')
    
    prevRpart = predict(Modelo, data = dados, type="class")
    
    del rpart
    del utils
    del base
    del Modelo
    del predict
    
    prevRpart.levels[prevRpart[0]-1]

In [8]:
base = importr('base')
utils = importr('utils')

robjects.r['load']('Modelos/df.100x1x100.Ocorrencias.Rdata')
dados_df = robjects.r['dados.df']

with localconverter(robjects.default_converter + pandas2ri.converter):
  df = robjects.conversion.rpy2py(dados_df)

In [9]:
display(df)

Status  \
ddb39e0b30270c3bf27db6d62ad5871b        c("BIOLOGIA ANIMAL", "CIENCIAS BIOLOGICAS")   
d58166e6fca41b68767fb802eb141fc5  c("PLANEJAMENTO TERRITORIAL E DESENVOLVIMENTO ...   
4598f020285da57279424db16205c63b           c("FARMACOQUIMICA", "CIENCIAS DA SAUDE")   
bbc628e8b71a662acb50d1bfcced2db5            c("FITOSSANIDADE", "CIENCIAS AGRARIAS")   
7789420b868d0ad2bd2b7aa04a6b571f   c("RELACOES INTERNACIONAIS", "CIENCIAS HUMANAS")   
...                                                                             ...   
2116f3002ca1971a2cdc7db605be1610  c("TEORIA ECONOMICA", "CIENCIAS SOCIAIS APLICA...   
29aaf676b3c769d6009d82dc1d69eb10     c("COMUNICACAO", "CIENCIAS SOCIAIS APLICADAS")   
402636af64727c16680338ade1ddd048          c("SOLO E AMBIENTE", "CIENCIAS AGRARIAS")   
eef474adc4c2d494dca53fa6b3bd8211  c("PRATICAS SOCIAIS EM ENFERMAGEM E SAUDE", "C...   
3742e50c7bb616cd77d313fdc2485e1e           c("SISTEMAS DE PRODUCAO", "ENGENHARIAS")   

                                  v1075  v592  v1341  v2630  v101  v292  \
ddb39e0b30270c3bf27db6d62ad5871b    0.0   0.0    0.0    1.0   1.0   0.0   
d58166e6fca41b68767fb802eb141fc5    0.0   0.0    0.0    0.0   0.0   0.0   
4598f020285da57279424db16205c63b    0.0   0.0    0.0    0.0   0.0   0.0   
bbc628e8b71a662acb50d1bfcced2db5    0.0   0.0    0.0    0.0   0.0   0.0   
7789420b868d0ad2bd2b7aa04a6b571f    0.0   0.0    0.0    0.0   0.0   0.0   
...                                 ...   ...    ...    ...   ...   ...   
2116f3002ca1971a2cdc7db605be1610    0.0   0.0    0.0    0.0   1.0   0.0   
29aaf676b3c769d6009d82dc1d69eb10    0.0   0.0    0.0    0.0   0.0   0.0   
402636af64727c16680338ade1ddd048    2.0   1.0    0.0    0.0   1.0   0.0   
eef474adc4c2d494dca53fa6b3bd8211    0.0   2.0    0.0    0.0   0.0   0.0   
3742e50c7bb616cd77d313fdc2485e1e    0.0   4.0    0.0    0.0   0.0   0.0   

                                  v5921  v42  v1349  ...  v12657  v1222  \
ddb39e0b30270c3bf27db6d62ad5871b    0.0  2.0    0.0  ...     0.0    0.0   
d58166e6fca41b68767fb802eb141fc5    0.0  0.0    0.0  ...     0.0    0.0   
4598f020285da57279424db16205c63b    0.0  0.0    0.0  ...     0.0    0.0   
bbc628e8b71a662acb50d1bfcced2db5    0.0  0.0    0.0  ...     0.0    0.0   
7789420b868d0ad2bd2b7aa04a6b571f    0.0  0.0    0.0  ...     0.0    0.0   
...                                 ...  ...    ...  ...     ...    ...   
2116f3002ca1971a2cdc7db605be1610    0.0  1.0    0.0  ...     0.0    0.0   
29aaf676b3c769d6009d82dc1d69eb10    0.0  2.0    0.0  ...     0.0    0.0   
402636af64727c16680338ade1ddd048    0.0  0.0    0.0  ...     0.0    0.0   
eef474adc4c2d494dca53fa6b3bd8211    0.0  2.0    0.0  ...     0.0    0.0   
3742e50c7bb616cd77d313fdc2485e1e    0.0  0.0    0.0  ...     0.0    0.0   

                                  v14992  v2112  v6086  v22018  v3773  v4492  \
ddb39e0b30270c3bf27db6d62ad5871b     0.0    0.0    0.0     0.0    0.0    0.0   
d58166e6fca41b68767fb802eb141fc5     0.0    0.0    0.0     0.0    0.0    0.0   
4598f020285da57279424db16205c63b     0.0    0.0    0.0     0.0    0.0    0.0   
bbc628e8b71a662acb50d1bfcced2db5     0.0    0.0    0.0     0.0    0.0    0.0   
7789420b868d0ad2bd2b7aa04a6b571f     0.0    0.0    0.0     0.0    0.0    0.0   
...                                  ...    ...    ...     ...    ...    ...   
2116f3002ca1971a2cdc7db605be1610     0.0    0.0    0.0     0.0    0.0    0.0   
29aaf676b3c769d6009d82dc1d69eb10     0.0    0.0    0.0     0.0    0.0    0.0   
402636af64727c16680338ade1ddd048     0.0    0.0    0.0     0.0    0.0    0.0   
eef474adc4c2d494dca53fa6b3bd8211     0.0    0.0    0.0     0.0    0.0    0.0   
3742e50c7bb616cd77d313fdc2485e1e     0.0    0.0    0.0     0.0    0.0    0.0   

                                  v3677  v9307  
ddb39e0b30270c3bf27db6d62ad5871b    0.0    0.0  
d58166e6fca41b68767fb802eb141fc5    0.0    0.0  
4598f020285da57279424db16205c63b    0.0    0.0  
bbc628e8b71a662acb50d1bfcced2db5    0.0    0.0  
7789420b868d0ad2bd2b7aa0

In [46]:
entrada = df.loc[['eef474adc4c2d494dca53fa6b3bd8211']]
del entrada['Status']

display(entrada)

v1075  v592  v1341  v2630  v101  v292  \
eef474adc4c2d494dca53fa6b3bd8211    0.0   2.0    0.0    0.0   0.0   0.0   

                                  v5921  v42  v1349  v934  ...  v12657  v1222  \
eef474adc4c2d494dca53fa6b3bd8211    0.0  2.0    0.0   0.0  ...     0.0    0.0   

                                  v14992  v2112  v6086  v22018  v3773  v4492  \
eef474adc4c2d494dca53fa6b3bd8211     0.0    0.0    0.0     0.0    0.0    0.0   

                                  v3677  v9307  
eef474adc4c2d494dca53fa6b3bd8211    0.0    0.0  

[1 rows x 3135 columns]

In [47]:
area = prevNN(entrada)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


--------------------------  -----------------------------
H2O_cluster_uptime:         2 hours 10 mins
H2O_cluster_timezone:       America/Sao_Paulo
H2O_data_parsing_timezone:  UTC
H2O_cluster_version:        3.38.0.1
H2O_cluster_version_age:    7 months and 29 days !!!
H2O_cluster_name:           H2O_from_python_renan_dcg5nh
H2O_cluster_total_nodes:    1
H2O_cluster_free_memory:    3.674 Gb
H2O_cluster_total_cores:    8
H2O_cluster_allowed_cores:  8
H2O_cluster_status:         locked, healthy
H2O_connection_url:         http://localhost:54321
H2O_connection_proxy:       {"http": null, "https": null}
H2O_internal_security:      False
Python_version:             3.10.6 final
--------------------------  -----------------------------

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


In [48]:
area

'CIENCIAS DA SAUDE'

In [49]:
modelos = get_best_models(area)

In [50]:
display(modelos)

Tipo Subtipo  MinDocs RangeDocs  count
0   RF  ranger      200     1x320     36
1   RF  ranger      200     1x280     34
2   RF    trad      200     1x280     25

In [15]:
with localconverter(robjects.default_converter + pandas2ri.converter):
      dados = robjects.conversion.py2rpy(entrada)

In [16]:
subareas = []

In [26]:
for i in modelos.index:
    str_modelo = "Modelo." + str(modelos['Tipo'][i]) + "." + str(modelos['Subtipo'][i]) + "." + str(modelos['MinDocs'][i]) + "x" + str(modelos['RangeDocs'][i]) + ".Ocorrencias.ResearchAreaSA." + area + ".RData"
    
    if(str(modelos['Tipo'][i]) == "RF" and str(modelos['Subtipo'][i]) == "ranger"):
        prev_sub =  prevRFranger(dados, str_modelo)
        subareas.append(prev_sub)
    
    if(str(modelos['Tipo'][i]) == "RF" and str(modelos['Subtipo'][i]) == "trad"):
        str_modelo = "Modelo." + str(modelos['Tipo'][i]) + "." + str(modelos['Subtipo'][i]) + "." + str(modelos['MinDocs'][i]) + "x" + str(modelos['RangeDocs'][i]) + ".Ocorrencias.RData"
        
        prev_sub =  prevRFtrad(dados, str_modelo)
        subareas.append(prev_sub
    
    if(str(modelos['Tipo'][i]) == "C50"):
        prev_sub =  prevC50(dados, str_modelo)
        subareas.append(prev_sub)
        
    if(str(modelos['Tipo'][i]) == "RPART"):
        prev_sub =  prevRpart(dados, str_modelo)
        subareas.append(prev_sub)

R[write to console]: Error in predict.ranger.forest(forest, data, predict.all, num.trees, type,  : 
  Error: One or more independent variables not found in data.

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  libraries ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ contain no packages

R[write to console]: 2: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  libraries ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ contain no packages

R[write to console]: 3: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  libraries ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-lib

RRuntimeError: Error in predict.ranger.forest(forest, data, predict.all, num.trees, type,  : 
  Error: One or more independent variables not found in data.


In [27]:
print(subareas)

['c("DIREITO, ESTADO E CONSTITUICAO", "CIENCIAS SOCIAIS APLICADAS")']
